# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [2]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached smdebug-1.0.12-py2.py3-none-any.whl (270 kB)
  Using cached pyinstrument-3.4.2-py2.py3-none-any.whl (83 kB)
  Using cached pyinstrument_cext-0.2.4-cp37-cp37m-manylinux2010_x86_64.whl (20 kB)
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [1]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [4]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip


--2022-02-26 18:29:42--  https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 3.5.160.129
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|3.5.160.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1132023110 (1.1G) [application/zip]
Saving to: ‘dogImages.zip.2’

dogImages.zip.2     100%[===================>]   1.05G  42.2MB/s    in 34s     

2022-02-26 18:30:19 (31.8 MB/s) - ‘dogImages.zip.2’ saved [1132023110/1132023110]

Archive:  dogImages.zip
replace dogImages/test/001.Affenpinscher/Affenpinscher_00003.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [2]:
#Torch vision download
!pip install torchvision --no-cache-dir

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [3]:
#imports

import sagemaker
import torchvision

from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
from sagemaker.pytorch import PyTorch
from torchvision import transforms

In [27]:
#TODO: Declare your HP ranges, metrics etc.
objective_metric_name = "Accuracy"
objective_type = "Maximize"
metric_definitions = [{"Name": "Accuracy", "Regex": "Test set: Accuracy: ([0-9\\.]+)"}]
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch-size": CategoricalParameter([32, 64, 128, 256, 512]),
    "epochs": IntegerParameter(2, 4)
}

In [37]:
#TODO: Create estimators for your HPs

roleget = sagemaker.get_execution_role()

estimator = PyTorch(
    entry_point="hpo.py",
    role=roleget,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.c5.4xlarge"
)
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

In [38]:
sagemaker_session = sagemaker.Session()
bucket= "dogproj"
inputs = sagemaker_session.upload_data(path="dogImages", bucket=bucket, key_prefix="Images")

In [ ]:
# TODO: Fit your HP Tuner
import os
os.environ['SM_CHANNEL_TRAINING']='s3://dogproj/Images/'
os.environ['SM_MODEL_DIR']='s3://dogproj/Images/model/'
os.environ['SM_OUTPUT_DATA_DIR']='s3://dogproj/Images/Out/'
tuner.fit({"training": "s3://dogproj/Images/"}) # TODO: Remember to include your data channels

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [47]:


# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator() #TODO

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()


2022-03-07 20:19:00 Starting - Preparing the instances for training
2022-03-07 20:19:00 Downloading - Downloading input data
2022-03-07 20:19:00 Training - Training image download completed. Training in progress.
2022-03-07 20:19:00 Uploading - Uploading generated training model
2022-03-07 20:19:00 Completed - Training job completed


{'_tuning_objective_metric': '"Accuracy"',
 'batch-size': '"256"',
 'epochs': '4',
 'lr': '0.0012316118290660798',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"pytorch-training-2022-03-07-18-20-32-419"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-2"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-2-669350239385/pytorch-training-2022-03-07-18-20-32-419/source/sourcedir.tar.gz"'}

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [49]:
# TODO: Set up debugging and profiling rules and hooks
from sagemaker.debugger import (
    Rule,
    DebuggerHookConfig,
    rule_configs,
)
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
]
hook_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "100", "eval.save_interval": "10"}
)

In [50]:
hyperparameters = {"epochs": "4", "batch-size": "256", "test-batch-size": "1000", "lr": "0.0012316118290660798"}

In [51]:
# TODO: Create and fit an estimator

estimator = PyTorch(
    entry_point="train_model.py",
    role=roleget,
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type="ml.c5.4xlarge"
)

In [58]:
import os
os.environ['SM_CHANNEL_TRAINING']='s3://dogproj/Images/'
os.environ['SM_MODEL_DIR']='s3://dogproj/Images/model/'
os.environ['SM_OUTPUT_DATA_DIR']='s3://dogproj/Images/Out/'
estimator.fit({"training": "s3://dogproj/Images/"})

2022-03-15 18:06:08 Starting - Starting the training job...
2022-03-15 18:06:23 Starting - Preparing the instances for trainingProfilerReport-1647367567: InProgress
......
2022-03-15 18:07:23 Downloading - Downloading input data.........
2022-03-15 18:09:00 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-03-15 18:08:57,101 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-03-15 18:08:57,103 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-15 18:08:57,110 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-03-15 18:08:57,115 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-03-15 18:08:57,450 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-15 18:08:57,

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()